## Fasam - NLP Competition

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

['df_train.csv', 'sampleSubmission.csv', 'df_valid.csv']


Esse notebook descreve o pipelie para utilização na competição do **Kaggle da Fasam**. Faz parte da avaliação prática dos alunos da turma de Deep Learning.


### Roteiro do Notebook

* Leitura do Dataset
* Criação do Modelo
* Avaliação e Criação do arquivo de submission.csv

### Problema


Uma revista precisa catalogar todas as suas notícias em diferentes categorias. O objetivo desta competição é desenvolver o melhor modelo de aprendizagem profunda para prever a categoria de novas notícias.


<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Untitled-Diagram.png
" style="width: 400px;"/>


As categorias possíveis são:

* ambiente
* equilibrioesaude
* sobretudo
* educacao
* ciencia
* tec
* turismo
* empreendedorsocial
* comida


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

sns.set(style="ticks")
warnings.filterwarnings("ignore")

In [3]:
# Bibliotecas do keras
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Leitura do dataset de treinamento

In [4]:
# Leitura do Dataset
df = pd.read_csv('../input/df_train.csv')
print(df.shape)
df.head()

(7894, 3)


,title,text,category
0,Casa da Barra Funda tem clima roceiro e receit...,"No Quem Quer Pão 75, casa de clima roceiro em ...",comida
1,Professores de SP decidem manter greve; grupo ...,Professores da rede estadual decidiram manter ...,educacao
2,"Em segunda edição, concurso paga R$ 35 mil par...",O Prêmio FedEx para Pequenas Empresas está com...,empreendedorsocial
3,Usar maconha por anos não faz tão mal para a s...,No esforço de entender os efeitos da maconha s...,equilibrioesaude
4,Baleia-azul percorre 5.200 km e revela a cient...,"Isabela, uma baleia-azul, ajudou os cientistas...",ciencia


Todos os artigos contêm o **título, descrição e link** da matéria original. Por último a categoria que pertence esse artigo.

In [5]:
## Definição de alguns parâmetros dos modelos e tokenização

# Tamanho da sequencia
seq_size     = 10

# Máximo de tokens 
max_tokens   = 2500

# Tamanho do embedding
embed_dim    = 128

Iremos utilizar o titulo para o nosso modelo baseline. O processo abaixo cria o **input** da nossa rede e prepara o **target**

In [6]:
## Utilizaremos apenas o .title (input) e o .category (target) da nossa rede
# Textos
text         = df['title'].values
tokenizer    = Tokenizer(num_words=max_tokens, split=' ')

# Transforma o texto em números
tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)  

# Cria sequencias de tamanho fixo (input: X)
X = pad_sequences(X, maxlen=seq_size)

In [7]:
# Categoriza o target "category" -> [0,..., 1] (output: y)
Y_classes = pd.get_dummies(df['category']).columns
Y         = pd.get_dummies(df['category']).values

In [8]:
(X.shape, Y.shape)

((7894, 10), (7894, 9))

## Criação do Modelo

Iremos utilizar uma RNN em um modelo simples.

In [9]:
def base_model():
    model = Sequential()
    
    # Embedding Layer
    model.add(Embedding(max_tokens, embed_dim, 
                        input_length = seq_size))
    # RNN Layer
    model.add(LSTM(seq_size))
    
    # Dense Layer
    model.add(Dense(len(Y_classes), activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', 
                  optimizer='adam',
                  metrics = ['accuracy'])
    
    model.summary()
    
    return model

base_model = base_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 128)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                5560      
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 99        
Total params: 325,659
Trainable params: 325,659
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Separa o dataset em dados de treinamento/teste
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, 
                                                      test_size = 0.20, 
                                                      random_state = 42)

# Treina o modelo
hist = base_model.fit(X_train, Y_train, 
              validation_data =(X_valid, Y_valid),
              batch_size=300, nb_epoch = 3,  verbose = 1)

Train on 6315 samples, validate on 1579 samples
Epoch 1/3
6315/6315 [==============================] - 2s 288us/step - loss: 2.1766 - acc: 0.2559 - val_loss: 2.1460 - val_acc: 0.3186
Epoch 2/3
6315/6315 [==============================] - 0s 78us/step - loss: 2.0986 - acc: 0.3379 - val_loss: 2.0508 - val_acc: 0.3331
Epoch 3/3
6315/6315 [==============================] - 0s 76us/step - loss: 1.9482 - acc: 0.3525 - val_loss: 1.8771 - val_acc: 0.3597


## Avaliação e Criação do arquivo de submission.csv

In [11]:
# Avaliação do modelo para o dataset de test

val_loss, val_acc = base_model.evaluate(X_valid, Y_valid)

print('A acurácia do modelo está de: '+str(val_acc*100)+'%')

1579/1579 [==============================] - 0s 61us/step
A acurácia do modelo está de: 35.97213426974093%


### Criando arquivo de submission para o Kaggle

In [12]:
# Leitura do Dataset de validação dos resultados
df_valid = pd.read_csv('../input/df_valid.csv')
print(df_valid.shape)
df_valid.head()

(4251, 3)


,Unnamed: 0,title,text
0,0,"Vitrine de Dilma, Pronatec terá orçamento 65% ...","Em compasso de espera, o baiano Leonardo Souza..."
1,1,"Por direitos autorais e publicidade, 'youtuber...",Fosse o segmento de vídeos on-line um tribunal...
2,2,Rótulos de alimentos terão que alertar sobre l...,Rótulos de todos os alimentos e bebidas indust...
3,3,Sociedade britânica de compositores processa S...,"A PRS for Music, maior cooperativa de direitos..."
4,4,"Por Fies, aluna madruga na porta da FMU, mas s...","Rafhaela Melo, 19, aluna do curso de economia ..."


O dataset de validação, o que será utilizado para calcular o Ranking no Kaggle, contêm apenas as informações de Título e Texto do arquivo.  O modelo criado deve ser capaz de classificar em qual das categorias esse artigo pertence

In [13]:
def predict(text):
    '''
    Utiliza o modelo treinado para realizar a predição
    '''
    new_text = tokenizer.texts_to_sequences(text)
    new_text = pad_sequences(new_text, maxlen=seq_size)
    pred     = base_model.predict_classes(new_text)#[0]
    return pred

In [14]:
# Como utilizamos o titulo no treinamento, iremos utilizar o titulo na predição também

pred         = predict(df_valid.title)
pred_classes = [Y_classes[c] for c in pred]
pred_classes[:5]

['educacao', 'tec', 'educacao', 'tec', 'educacao']

In [15]:
# Atualizando a categoria dos artigos no dataset de validação
df_valid['category'] = pred_classes
df_valid.head()

,Unnamed: 0,title,text,category
0,0,"Vitrine de Dilma, Pronatec terá orçamento 65% ...","Em compasso de espera, o baiano Leonardo Souza...",educacao
1,1,"Por direitos autorais e publicidade, 'youtuber...",Fosse o segmento de vídeos on-line um tribunal...,tec
2,2,Rótulos de alimentos terão que alertar sobre l...,Rótulos de todos os alimentos e bebidas indust...,educacao
3,3,Sociedade britânica de compositores processa S...,"A PRS for Music, maior cooperativa de direitos...",tec
4,4,"Por Fies, aluna madruga na porta da FMU, mas s...","Rafhaela Melo, 19, aluna do curso de economia ...",educacao


In [16]:
def create_submission(df):
    f = open('submission_valid.csv', 'w')
    f.write('id,category\n')
    for i, row in df.iterrows():
        f.write('{},{}\n'.format(i, row.category))
    f.close()
    
# Criando o arquivo submission_valid.csv contendo os dados para cálculo do raning no kaggle
# Esse arquivo deve ser enviado para o kaggle
create_submission(df_valid)    